In [1]:
#import solcx
#solcx.install_solc('0.8.28')

In [2]:
#%pip install ape-solidity

In [3]:
#%pip install eth-tester

In [4]:
import shutil
import time
import pandas as pd

In [5]:
from ape import project
from ape import Project
from ape import networks

from ape import chain
from ape import accounts

from ape.logging import logger, LogLevel


logger.set_level(LogLevel.WARNING)
sender = accounts.test_accounts[0]

In [6]:
#orig = Project('.')
#a = orig.get_contract('PIController.sol')
try:
    shutil.rmtree("src/.ipynb_checkpoints")
except:
    print("can't delete")
    pass
old_pi = project.get_contract("PIController") 

can't delete


In [7]:
new_pi = project.get_contract("PIControllerNew") 

In [8]:
rpc = 'foundry'

In [9]:
kp = 222002205862
ki = 25000

kp = 222002205862000000
ki = 250000000000000000

coBias = 0
perSecondIntegralLeak = 999997208243937652252849536 #1% per hour
#perSecondIntegralLeak = int(10**27)
outputUpperBound = 18640000000000000000
outputLowerBound = -51034000000000000000

importedState = [0,0,0]
with networks.ethereum.local.use_provider(rpc):
    old_pi_c = old_pi.deploy(
            b'test control variable',
            kp,
            ki,
            coBias,
            perSecondIntegralLeak,
            outputUpperBound,
            outputLowerBound,
            importedState,
            sender=sender
          )

In [10]:
with networks.ethereum.local.use_provider(rpc):
    new_pi_c = new_pi.deploy(
            b'test control variable',
            kp,
            ki,
            coBias,
            perSecondIntegralLeak,
            outputUpperBound,
            outputLowerBound,
            importedState,
            sender=sender
          )

In [11]:
with networks.ethereum.local.use_provider(rpc):
    new_pi_c.modifyParametersAddr('seedProposer',  sender, sender=sender)

In [12]:
# not workinging, so commented out seedProposer requirement
#with networks.ethereum.local.use_provider("test"):
#    old_pi_c.modifyParameters(b"seedProposer",  sender, sender=sender)

In [13]:
n = 20000
old_values = []
new_values = []

In [14]:
with networks.ethereum.local.use_provider(rpc):
    chain.provider.auto_mine = False
    for i in range(n):
        receipt_old = old_pi_c.update(int(1/100*1e27), sender=sender)
        chain.mine(1, timestamp=chain.blocks.head.timestamp + 12)
        old_values.append(list(receipt_old.return_value) + [chain.blocks.head.timestamp])


In [15]:
with networks.ethereum.local.use_provider(rpc):
    chain.provider.auto_mine = False
    for i in range(n):
        receipt_new = new_pi_c.update(int(1/100*1e27), sender=sender)
        chain.mine(1, timestamp=chain.blocks.head.timestamp + 12)
        new_values.append(list(receipt_new.return_value) + [chain.blocks.head.timestamp])

In [16]:
df_new = pd.DataFrame(new_values, columns = ['output_new', 'p_output_new', 'i_output_new', 'ts_new'])
df_old = pd.DataFrame(old_values, columns = ['output_old', 'p_output_old', 'i_output_old', 'ts_old'])
#df = pd.DataFrame(values, columns = ['error_integral', 'error'])

In [17]:
df_new.to_csv('new.csv.gz', index=False)
df_old.to_csv('old.csv.gz', index=False)

In [18]:
all_df = pd.concat((df_new, df_old), axis=1)

In [19]:
all_df.head(10)

,output_new,p_output_new,i_output_new,ts_new,output_old,p_output_old,i_output_old,ts_old
0,18640000000000000000,2220022058620000201127263,0,1736094013,18640000000000000000,2220022058620000201127263,0,1735854013
1,18640000000000000000,2220022058620000201127263,30000000000000002717908992,1736094025,18640000000000000000,2220022058620000201127263,30000000000000002717908992,1735854025
2,18640000000000000000,2220022058620000201127263,59998994983249342425491297,1736094037,18640000000000000000,2220022058620000201127263,59998994983249342425491297,1735854037
3,18640000000000000000,2220022058620000201127263,59997990000167301718939124,1736094049,18640000000000000000,2220022058620000201127263,59997990000167301718939124,1735854049
4,18640000000000000000,2220022058620000201127263,59996985050752755398515228,1736094061,18640000000000000000,2220022058620000201127263,59996985050752755398515228,1735854061
5,18640000000000000000,2220022058620000201127263,59995980135004575584359246,1736094073,18640000000000000000,2220022058620000201127263,59995980135004575584359246,1735854073
6,18640000000000000000,2220022058620000201127263,59994975252921634434395420,1736094085,18640000000000000000,2220022058620000201127263,59994975252921634434395420,1735854085
7,18640000000000000000,2220022058620000201127263,59993970404502804144331330,1736094097,18640000000000000000,2220022058620000201127263,59993970404502804144331330,1735854097
8,18640000000000000000,2220022058620000201127263,59992965589746956947656630,1736094109,18640000000000000000,2220022058620000201127263,59992965589746956947656630,1735854109
9,18640000000000000000,2220022058620000201127263,59991960808652965115641783,1736094121,18640000000000000000,2220022058620000201127263,59991960808652965115641783,1735854121


In [20]:
all_df['output_new'].equals(all_df['output_old'])

True

In [21]:
all_df['p_output_new'].equals(all_df['p_output_old'])

True

In [22]:
all_df['i_output_new'].equals(all_df['i_output_old'])

True

In [23]:
len(all_df)

20000